In [1]:
import os
import time

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import List, Dict
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0.7)

In [4]:
# DEFINE BASE AGENTS CLASS
class Agent:
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills
        self.llm = llm

    def process(self, task: str, context: List[Dict] = None) -> str:
        messages = [
            SystemMessage(content=f"You are {self.name}, a {self.role}. Your skills includes: {', '.join(self.skills)}. Respond to the task absed on your role and skills.")
        ]

        if context:
            for msg in context:
                if msg['role'] == 'human':
                    messages.append(HumanMessage(content=msg['content']))
                elif msg['role'] == 'ai':
                    messages.append(AIMessage(content=msg['content']))

        messages.append(HumanMessage(content=task))
        response = self.llm.invoke(messages)
        return response.content

In [5]:
# define specialized agents: HistoryResearchAgent and DataAnalysisAgent
class HistoryResearchAgent(Agent):
    def __init__(self):
        super().__init__("Clio", "History Research Specialist", ["deep knowledge of historical events", "understanding of historical contexts", "indetifying historical trends"])

class DataAnalysisAgent(Agent):
    def __init__(self):
        super().__init__('Data', "Data Analysis Expert", ["interpreting numerical data", "statstical analysis", "data visualization description"])

In [6]:
# Define the different functions for the collab system

# Research historical context
def research_historical_context(history_agent, task: str, context: list) -> list:
    print("Hsitory Agent: Researching historical context..")
    history_task = f"Provide relevant historical context and information for the following task: {task}"
    history_result = history_agent.process(history_task)
    context.append({"role": "ai", "content": f"History Agent: {history_result}"})
    print(f"Historical context provided: {history_result[:100]}...\n")
    return context

In [7]:
# Identify data needs

def identify_data_needs(data_agent, task: str, context: list) -> list:
    print("Data Agent: Identifying data needs based on historical context...")
    historical_context = context[-1]["content"]
    data_need_task = f"Based on the historical context, what specific data or statistical information would be helpful to answer the original question? Historical context: {historical_context}"
    data_need_result = data_agent.process(data_need_task, context)
    context.append({"role": "ai", "content": f"Data Agent: {data_need_result}"})
    print(f"Data needs identified: {data_need_result[:100]}...\n")
    return context

In [8]:
# Provide historical Data

def provide_historical_data(history_agent, task: str, context: list) -> list:
    print("History agentL prividing relevant historical data...")
    data_needs = context[-1]["content"]
    data_provision_task = f"based on the data needs identified, provide the relevant historical data or statistics. Data needs: {data_needs}"
    data_provision_result = history_agent.process(data_provision_task, context)
    context.append({"role": "ai", "content": f"History Agent: {data_provision_result}"})
    print(f"Historical data provided: {data_provision_result[:100]}...\n")
    return context

In [9]:
# Analyze Data

def analyze_data(data_agent, task: str, context: list) -> list:
    print("Data Agent: Analyzing historical data...")
    historical_data = context[-1]["content"]
    analysis_task = f"analyze the historical data provided and describe any trends or insights relevant to the original task. Historical data: {historical_data}"
    analysis_result = data_agent.process(analysis_task, context)
    context.append({"role":"ai","content":f"Data Agent: {analysis_result}"})
    print(f"Data analysis results: {analysis_result[:100]}...\n")
    return context

In [10]:
# Synthesize final answer
def synthesize_final_answer(history_agent, task: str, context: list) -> str:
    print("History Agent: Synthesizing final answer...")
    synthesize_task = "Based on all historical context, data, and analysis, provide a comprehensive answer to the original task"
    final_result = history_agent.process(synthesize_task, context)
    return final_result

In [ ]:
class HistoryDataCollaborationSystem:
    def __init__(self):
        self.history_agent = Agent("Clio", "History Research Specialist", ["deep knowledge of historical events", "understanding of historical contexts", "identifying historical trends"])
        self.data_agent = Agent("Data", "Data Analysis Expert", ["interpreting numerical data", "statistical analysis", "data visualization description"])

    def solve(self, task: str, timeout: int = 300) -> str:
        print(f"\nStarting collaboration to solve: {task}\n")
        
        start_time = time.time()
        context = []
        
        steps = [
            (research_historical_context, self.history_agent),
            (identify_data_needs, self.data_agent),
            (provide_historical_data, self.history_agent),
            (analyze_data, self.data_agent),
            (synthesize_final_answer, self.history_agent)
        ]
        
        for step_func, agent in steps:
            if time.time() - start_time > timeout:
                return "Operation timed out. The process took too long to complete."
            try:
                result = step_func(agent, task, context)
                if isinstance(result, str):
                    return result  # This is the final answer
                context = result
            except Exception as e:
                return f"Error during collaboration: {str(e)}"
        
        print("\nCollaboration complete. Final answer synthesized.\n")
        return context[-1]["content"]

In [15]:
# USAGE

collaboration_system = HistoryDataCollaborationSystem()

# Define a complex historical question that requires both historical knowledge and data anlysis
question = "How did urbanization rates in Europe compare to those in North America during the Industrial Revolution, and what were the main factors influencing these trends?"

# Solve the question using the collab system
result = collaboration_system.solve(question)

# Print the result

with open("collab_result.txt", "w") as f:
    f.write(result)

print(result)


👥 Starting collaboration to solve: How did urbanization rates in Europe compare to those in North America during the Industrial Revolution, and what were the main factors influencing these trends?

Hsitory Agent: Researching historical context..
Historical context provided: The Industrial Revolution, which began in the late 18th century and continued into the 19th century,...

Data Agent: Identifying data needs based on historical context...
Data needs identified: To effectively analyze and compare the urbanization patterns during the Industrial Revolution in Eur...

History agentL prividing relevant historical data...
Historical data provided: Certainly! Below is a collection of historical data and statistics relevant to the urbanization patt...

Data Agent: ANalyzing historical data...
Data analysis results: Data Agent: Analyzing the historical data on urbanization during the Industrial Revolution reveals s...

History Agent: Synthesizing final answer...
Certainly! Here’s a comprehe